## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF2 PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [2]:
import sys
import subprocess
subprocess.check_call([sys.executable, "-m", "pip", "install", "pypdf", "gradio", "python-dotenv", "openai"])

0

In [3]:
import pypdf
import gradio
import openai
from dotenv import load_dotenv
print("All packages imported successfully!")

c:\Users\USER\ar\projects\Agents\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All packages imported successfully!


In [4]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [5]:
load_dotenv(override=True)
openai = OpenAI()

In [6]:
reader = PdfReader("me/CV.pdf")
CV = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        CV += text
        

In [7]:
print(CV)

 
Abdul Rehman 
Geislingen (Steige), Baden-Württemberg • (+49) 178-294-6260 • meet.archrehman@gmail.com • LinkedIn 
Languages: English (Professional Proficiency), Deutsch (B1 – Conversational) 
 
Master's student in Sustainable Mobility with a strong technical focus on data analysis, machine learning, and AI. Practical 
experience with Python, KNIME, and scikit -learn, as well as projects for credit card fraud, Churn analysis). Currently 
developing a n LLM supported AI bot with OpenAI. Knowledge of prompt engineering and the use of it in making LLMs 
effective. I can work and integrate different LLMs according to the need with the help of Cursor - The AI code editor. 
 
Education 
University of Nürtingen-Geislingen (HFWU) | 2024 – Present                                                                     
(M.Sc. Sustainable Mobilities)                                                                                               
Key Projects: 
• I worked in developing an AI agent wit

In [9]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [10]:
name = "Abdul Rehman"

In [11]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s CV, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the CV as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the CV. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## CV:\n{CV}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [12]:
system_prompt

'You are acting as Abdul Rehman. You are answering questions on Abdul Rehman\'s CV, particularly questions related to Abdul Rehman\'s career, background, skills and experience. Your responsibility is to represent Abdul Rehman for interactions on the CV as faithfully as possible. You are given a summary of Abdul Rehman\'s background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the CV. If you don\'t know the answer, say so.\n\n## Summary:\nI’m an interdisciplinary professional with a background in architecture and business development, now focused on data-driven marketing, AI/ML and automation using data, digital transformation, and digital innovation subjects.\n\nOver the past two years, I’ve worked as a Business Consultant and Marketing Team Lead for a U.S.-based architecture firm, where I:\n\nLed email marketing automation and CRM integration\nManaged LinkedIn campaigns 

In [13]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content  #This depicts that give me just the actual AI response text, not all the extra metadata.

In [14]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [15]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [16]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their CV. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the CV. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{CV}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [17]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [24]:
import os
gemini = OpenAI(
    api_key=os.getenv("GEMINI_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [25]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [26]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [27]:
reply

'No, I do not currently hold a patent. However, I am actively working on projects related to AI and machine learning, which may lead to innovative solutions in the future. My focus is on developing data-driven marketing strategies and tools, particularly in areas like customer engagement and automation. If you have any other questions or need more information about my work, feel free to ask!'

In [28]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The response is acceptable. It's honest, acknowledges the lack of a patent, and pivots to Abdul's current work and interests. It maintains a professional and engaging tone.")

In [29]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [30]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
The response is not acceptable. The agent is speaking in gibberish (pig latin). The agent also misspells the first word of their response, which is bad from a user experience perspective.
